In [1]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [2]:
types_y=["Invest","Maintain","Sell"]
n = 50
k=len(types_y)
set_I = range(n)
set_k=range(k)


In [4]:
#NPV
path='C:\\Users\\hisha\\Desktop\\College\\GradSchool\\FALL\\MGSC662 Decision\\Assignments\\Ass2\\Excel2\\NPV.csv'
df=pd.read_csv(path)
df
df1=np.array(df)
df1
n=len(df)

print(df1)
#CashFlowNxtYr

path2='C:\\Users\\hisha\\Desktop\\College\\GradSchool\\FALL\\MGSC662 Decision\\Assignments\\Ass2\\Excel2\\CashFlowNxtYear.csv'
CashNxtYr=pd.read_csv(path2)
CashNxtYr
CashNxtYr1=np.array(CashNxtYr)
print(CashNxtYr1)

#Investment
path3='C:\\Users\\hisha\\Desktop\\College\\GradSchool\\FALL\\MGSC662 Decision\\Assignments\\Ass2\\Excel2\\InvestReq.csv'
Investment=pd.read_csv(path3)
Investment
Investment=np.array(Investment)
print(Investment)


[[-229  -44 -309]
 [-257  -16 1100]
 [1089  799  345]
 [ 681 -383  628]
 [-475  682  605]
 [-248  319  858]
 [-371  649  160]
 [-104  264 1064]
 [-332 1240  532]
 [ 344   19  417]
 [-169 1029 1061]
 [ 464 1056  459]
 [ 781  164  121]
 [ 950  136  223]
 [-233 -184 -547]
 [ 225 1231   66]
 [ 855 -241 -245]
 [ 257 -275  -80]
 [ 571   42  347]
 [ 450  -38  647]
 [-371 -410  300]
 [1131 -338  407]
 [ 692 1091   68]
 [1098 -420  226]
 [ 799  770 1080]
 [1247  850 1048]
 [ 551  195 -179]
 [-244  488  869]
 [ 280  240 -304]
 [ 701  298  232]
 [-145  237  615]
 [ 352  437 1102]
 [ -18  164  854]
 [ 193 -366 1165]
 [ 157  418 -528]
 [ 988 -332 1215]
 [1216 1186 -107]
 [ 607 -519  296]
 [ 255  998 -243]
 [ 422 -526   54]
 [ 599 -108 1033]
 [1125  -28  499]
 [-335 -209 -241]
 [ 697  389 -453]
 [ 719  -14 -533]
 [ -67  694  -68]
 [ 598  194 -409]
 [ 687  906 1173]
 [ 191  709  702]
 [1165  511  818]]
[[-300  350 -500]
 [-200   35 1600]
 [1200  900  400]
 [ 900 -700  800]
 [-600  900 1000]
 [  12   

In [5]:
#Model, Decision Variables, and OBJ
model = gp.Model("Assignment 2")
## Model before A-F constraints
#Decision Variables

x = model.addVars(50,3, lb=0, vtype= GRB.BINARY, name = ["x_"+str(i)+"_"+str(types_y[k]) for k in set_k for i in set_I])
x

obj=sum(x[i,k]*df1[i,k] for k in set_k for i in set_I)
model.setObjective(obj, GRB.MAXIMIZE)


Restricted license - for non-production use only - expires 2022-01-13


<gurobi.Constr *Awaiting Model Update*>

In [7]:

#Constraint for Investment and Cash flow

#ignore commented constraint,
#model.addConstr(sum(x[i,k]*df1[i,k] for k in set_k for i in set_I)>=sum(x[i,k]*CashNxtYr1[i,k] for k in set_k for i in set_I))

model.addConstr(sum(x[i,k]*Investment[i,k] for k in set_k for i in set_I)<=sum(x[i,k]*CashNxtYr1[i,k] for k in set_k for i in set_I))


#Constraint 1 of each
for i in range(50):
    model.addConstr(sum(x[i,k] for k in set_k)==1)
    

#model.optimize()
#get vars of the model
#model.getVars()

In [8]:
#A to F Constraints

#A
# =============================================================================
#Sell 1 Sell 2
model.addConstr((x[0,2]-x[1,0])<=0)
model.addConstr(x[1,0]-x[0,2]<=0)
# #sell 1 sell 3
model.addConstr(x[0,2]-x[2,2]<=0)
model.addConstr(x[2,2]-x[0,2]<=0)
##sell 1 then sell 4
model.addConstr(x[0,2]-x[3,2]<=0)
model.addConstr(x[3,2]-x[0,2]<=0)
# =============================================================================

#B
#invest 6 then invest 7
model.addConstr(x[5,0]-x[6,0]<=0)
model.addConstr(x[6,0]-x[5,0]<=0)

#C
#1 choice must be made
model.addConstr(x[2,2]+x[3,2]+x[4,2]+x[5,1]+x[6,2]+x[8,2]<=1)

#D
#at most 1
model.addConstr(x[3,0]+x[5,1]+x[1,2]+x[5,2]+x[7,2]+x[11,2]+x[13,0]<=1)

#E
#24 then also 28
model.addConstr(x[23,0]-x[27,0]<=0)

#F
#either or
model.addConstr(x[31,1]+x[31,0]<=x[29,1])


<gurobi.Constr *Awaiting Model Update*>

In [10]:
model.optimize()

Ass2 = 'Optimal' if GRB.OPTIMAL == 2 else 'Not optimal (others)'
print("="*30,"\nSolution Status:", Ass2)
print("Investment maxed: ${}".format(round(model.objVal),2))


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 76 rows, 150 columns and 508 nonzeros
Model fingerprint: 0xee64adb3
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 5 rows, 22 columns, 31 nonzeros

Continuing optimization...


Explored 1 nodes (1 simplex iterations) in 0.01 seconds
Thread count was 6 (of 6 available processors)

Solution count 4: 34036 33074 32351 25333 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.403600000000e+04, best bound 3.403600000000e+04, gap 0.0000%
Solution Status: Optimal
Investment maxed: $34036


In [11]:
#B
## Model Without constraints 
model1 = gp.Model("Assignment 2")

#Decision Variables

x = model1.addVars(50,3, lb=0, vtype= GRB.BINARY, name = ["x_"+str(i)+"_"+str(types_y[k]) for k in set_k for i in set_I])
x

obj=sum(x[i,k]*df1[i,k] for k in set_k for i in set_I)
model1.setObjective(obj, GRB.MAXIMIZE)


#Constraints
#model.addConstr(sum(x[i,k]*df1[i,k] for k in set_k for i in set_I)>=sum(x[i,k]*CashNxtYr1[i,k] for k in set_k for i in set_I))

model1.addConstr(sum(x[i,k]*Investment[i,k] for k in set_k for i in set_I)<=sum(x[i,k]*CashNxtYr1[i,k] for k in set_k for i in set_I))

#Constraint 1 of each
for i in range(50):
    model1.addConstr(sum(x[i,k] for k in set_k)==1)
    
model1.optimize()

Ass2 = 'Optimal' if GRB.OPTIMAL == 2 else 'Not optimal (others)'
print("="*30,"\nSolution Status:", Ass2)
print("Travel cost: ${}".format(round(model1.objVal),2))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 51 rows, 150 columns and 295 nonzeros
Model fingerprint: 0x5f9090fe
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 22526.000000
Presolve removed 51 rows and 150 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 6 available processors)

Solution count 2: 39625 22526 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.962500000000e+04, best bound 3.962500000000e+04, gap 0.0000%
Solution Status: Optimal
Travel cost: $39625


In [18]:
#C
#Model, Decision Variables, and OBJ
model3 = gp.Model("Assignment 2")
## Model before A-F constraints
#Decision Variables

x = model3.addVars(50,3, lb=0, vtype= GRB.BINARY, name = ["x_"+str(i)+"_"+str(types_y[k]) for k in set_k for i in set_I])
x

obj=sum(x[i,k]*df1[i,k] for k in set_k for i in set_I)
model3.setObjective(obj, GRB.MAXIMIZE)

#Constraint for Investment and Cash flow

#ignore commented constraint,
#model3.addConstr(sum(x[i,k]*df1[i,k] for k in set_k for i in set_I)>=sum(x[i,k]*CashNxtYr1[i,k] for k in set_k for i in set_I))

model3.addConstr(sum(x[i,k]*Investment[i,k] for k in set_k for i in set_I)<=sum(x[i,k]*CashNxtYr1[i,k] for k in set_k for i in set_I))


#Constraint 1 of each
for i in range(50):
    model3.addConstr(sum(x[i,k] for k in set_k)==1)
    

#model3.optimize()
#get vars of the model3
#model3.getVars()


#A to F Constraints

#A
# =============================================================================
#Sell 1 Sell 2
model3.addConstr((x[0,2]-x[1,0])<=0)
model3.addConstr(x[1,0]-x[0,2]<=0)
# #sell 1 sell 3
model3.addConstr(x[0,2]-x[2,2]<=0)
model3.addConstr(x[2,2]-x[0,2]<=0)
##sell 1 then sell 4
model3.addConstr(x[0,2]-x[3,2]<=0)
model3.addConstr(x[3,2]-x[0,2]<=0)
# =============================================================================

#B
#invest 6 then invest 7
model3.addConstr(x[5,0]-x[6,0]<=0)
model3.addConstr(x[6,0]-x[5,0]<=0)

#C
#1 choice must be made
model3.addConstr(x[2,2]+x[3,2]+x[4,2]+x[5,1]+x[6,2]+x[8,2]<=1)

#D
#at most 1
model3.addConstr(x[3,0]+x[5,1]+x[1,2]+x[5,2]+x[7,2]+x[11,2]+x[13,0]<=1)

#E
#24 then also 28
model3.addConstr(x[23,0]-x[27,0]<=0)

#F
#either or
model3.addConstr(x[31,1]+x[31,0]<=x[29,1])

model3.addConstr(sum(x[i,k]*df1[i,k] for k in set_k for i in set_I)<=34036-1)

model3.optimize()


Ass2 = 'Optimal' if GRB.OPTIMAL == 2 else 'Not optimal (others)'
print("="*30,"\nSolution Status:", Ass2)
print("Travel cost: ${}".format(round(model3.objVal),2))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 64 rows, 150 columns and 479 nonzeros
Model fingerprint: 0x5800b76f
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+04]
Found heuristic solution: objective 24506.000000
Presolve removed 16 rows and 30 columns
Presolve time: 0.00s
Presolved: 48 rows, 120 columns, 314 nonzeros
Found heuristic solution: objective 33984.000000
Variable types: 0 continuous, 120 integer (120 binary)

Root relaxation: objective 3.403500e+04, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 34035.0000    0    6 33984.0000 34035.0000  0.15%     -    0s
     0 